
# Texas Air Quality Notebook

**Welcome to our notebook on Texas Air Quality!** 

In this notebook we will be looking at Air Quality Index (AQI) scores in the surrounding Texas area. With so many pollutants in the air, especially as we head into the annual fire season, AQI becomes something we check on daily. For many of us, this AQI map is all too familiar. Throughout this module we will discuss how data can be used to visualize and uncover underlying trends in the world.

**Let's get started!**

<p align="center">
  <img src="images/Texas_AQI.png" width="600" height="600" align="center">
</p>

<br>

## Introduction to Jupyter Notebook

Before we get started with the data, let's talk about what Jupyter Notebook is. This lab is set up in a Jupyter Notebook. Notebooks can contain anything from live code, to written text, equations or visualizations. The content of notebooks are written into rectangular sections called **cells**. 

#### Types of Cells
There are two types of cells in Jupyter, **code** cells and **markdown** cells. **Code cells**, as you can imagine, contain code in Python, the programming language that we will be using throughout this notebook.  **Markdown cells**, such as this one, contain written text. You can select any cell by clicking on it one. 

#### Running Cells
'Running' a cell is similar to pressing 'Enter' on a calculator once you've typed in an expression; it computes all of the expressions contained within the cell.

To run a cell, you can do one of the following:

- press **Shift + Enter**
- click the **Run** button on the top tool bar

Running a markdown cell will embed the text into the notebook and running a code cell will evaluate the code and display its output under the cell. 

Let's try it! **Run the code cell below.**

In [1]:
print("Hello World!")

Hello World!


#### Editing and Saving

- To **edit** a cell, simply double click on the desired cell and begin typing. The cell that you are currently working in will be highlighted by a green box.
- To **save** the notebook, either click *Ctl + S* or navigate to the "File" dropdown and select "Save and Checkpoint"

#### Adding Cells
You can add a cell by clicking <b><code>Insert > Insert Cell Below</code></b> and choose the cell type in the drop down menu. Try adding a cell below to type in your name!


#### Deleting Cells 
To delete a cell, click on the <b><code>scissors</code></b> at the top or <b><code>Edit > Cut Cells</code></b>. Delete the cell below.

In [2]:
print("Delete this!")

Delete this!


**Important Tip**: Everytime you open a Jupyter notebook, it is extremely important to run all the cells from the beginning in order for the notebook to work. 

Now that we have had a brief crash course on Jupyter Notebooks, let's dive into Texas AQI!

<br>

## Introduction to the Data <a class="anchor" id="2"></a>

In this notebook we will look at data collected from PrupleAir, a company that manages a network of air quality sensors. The data from these sensors are then collected to create maps like the one displayed above that depicts an intuitive visualization of the air quality in a specific region. In the dataframe below, you will find several metrics that help us do this.

**Before we begin:**

- Click on <b><code>Cell</code></b> in the top toolbar  
- Click on <b><code>Run All</code></b> in the drop down
- Scroll back up to begin going through the notebook!

In [3]:
from IPython.display import HTML
hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input style="opacity:1" type="submit" value="Click here to reveal the raw code."></form>''')

In [4]:
hide_me
import matplotlib.pyplot as plt
import numpy as np
import purpleair
import folium
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from datetime import datetime
from IPython.display import clear_output


<br>

# PurpleAir Data

Before we begin looking at data collected from PurpleAir sensors, lets first take a look at what a sensor is, and what it measures. 


> Below is a picture of a real PurpleAir Air Quality Sensor. These sensor can be mounted both indoors or outdoors, and it tracks airborne particulate matter(PM) in real time using PMSX003 laser counters. Particulate matter can include things like dust, smoke, dirt and any other organic or inorganic particles in the air. With multiple sensors mounted in a region, PurpleAir can create a relatively accurate measure of AQI throughout the day as the air quality changes. 

For more information on how sensors work, take a look at the official PurpleAir website [here](https://www2.purpleair.com/community/faq#hc-what-do-the-numbers-on-the-purpleair-map-mean-1)!

<p align="center">
  <img src="images/purpleair-sensor-pm2.5.webp" width="" height="" align="center">
</p>

In order to work with the data, we need to pull it into our workspace. Fortunately, PurpleAir has created an API that allows users to pull in and work with their AQI data. In the code cell below we will import the prupleair API and use it to create a dataframe of data from all the PurpleAir sensors!

In [5]:
hide_me

from purpleair.network import SensorList
p = SensorList()
df = p.to_dataframe(sensor_filter='all',
                    channel='parent')

Initialized 22,490 sensors!


The dataframe below contains all the sensor data as of the latest update. It contains data on everything from the geograohical latitude and longitude of the sensor to data on the last time that sensor measured airborne PM.

In [6]:
hide_me

df

,parent,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,pressure,...,last_update_check,created,uptime,is_owner,10min_avg,30min_avg,1hour_avg,6hour_avg,1day_avg,1week_avg
id,,,,,,,,,,,,,,,,,,,,,
14633,None,37.275561,-121.964134,Hazelwood canary,outside,32.09,59.0,15.000000,57.0,1010.31,...,None,None,None,False,27.93,25.76,26.13,24.97,24.41,15.30
25999,None,30.053808,-95.494643,Villages of Bridgestone AQI,outside,14.50,69.0,20.555556,61.0,1011.57,...,None,None,None,False,15.58,15.67,15.95,13.14,10.62,14.16
14091,None,37.883620,-122.070087,WC Hillside,outside,23.15,59.0,15.000000,58.0,1005.19,...,None,None,None,False,22.41,21.27,19.50,23.36,29.93,23.15
108226,None,38.573703,-121.439113,"""C"" Street Air Shelter",inside,5.04,77.0,25.000000,45.0,1018.38,...,None,None,None,False,4.95,4.72,4.61,4.65,5.20,5.55
49409,None,18.759182,99.017172,"""First's Place""",outside,47.39,97.0,36.111111,26.0,983.11,...,None,None,None,False,48.45,52.95,57.63,55.00,44.61,31.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64085,None,36.785883,127.157040,청룡동행정복지센터,outside,49.94,58.0,14.444444,39.0,1027.62,...,None,None,None,False,48.45,47.34,51.95,82.21,68.14,44.54
64995,None,36.691324,126.585255,한서대학교,outside,42.81,71.0,21.666667,27.0,1017.76,...,None,None,None,False,41.62,38.88,35.13,36.26,35.59,27.62
64093,None,36.710720,126.548390,해미읍성,outside,40.09,65.0,18.333333,30.0,1028.44,...,None,None,None,False,41.41,39.74,39.59,43.08,40.54,31.21


Here is a breakdown of the dataframe above and what each column represents. 

`lat`: The latitude coordinate of the location

`lon`: The longitude coordinate of the location

`name`: The name of the location

`location_type`: The nature of the location (ie. inside or outside)

`pm_2.5`: The level of fine particulate matter in the air of that location

`temp_f`: The temperature of the location in degrees Farenheit 

`temp_c`: The temperature of the location in degrees Celsius 

`humidity`: The humidity percentage of the location

`pressure`: The pressure index of the location (in millibars)

`last_seen`: The last seen date and timestamp in UTC

`model`: Model of the specific sensor

`flagged`: Whether or not the channel was marked as flagged (usually based on a fault)

`age`: Sensor data age (when data was last received)

`10min_avg`: Average PM 2.5 AQI over the last 10 minutes 

`30min_avg`: Average PM 2.5 AQI over the last 30 minutes

`1hour_avg`: Average PM 2.5 AQI over the last hour

`6hour_avg`: Average PM 2.5 AQI over the last 6 hours

`1day_avg`: Average PM 2.5 AQI over the last day 

`1week_avg`: Average PM 2.5 AQI over the last week


<br>

### Airborne Particulate Matter (PM) 2.5 
While many of the column names are relatively straightforward, such as the "name" column (which displays the set name of the particular sensor), the "location_type" column (which indicates where it is an indoor or outdoor sensor), etc., we would like to draw your attention to the "pm_2.5" column. 

>The "pm_2.5" column represents the count of airborne pm that is larger than 2.5um/dl, in otherwords, airborne particles that have a diameter of 2.5 micrometers or less. In high levels, PM 2.5 particles can reduce visibility and cause the air to appear hazy. Tracking PM 2.5 is important because prolonged exposure to high levels of PM 2.5 particles can cause adverse US Environmental Protection Agency (EPA) use to calculate the local Air Quality Index (AQI).

If you go to the PurpleAir website [here](https://map.purpleair.com/1/mAQI/a10/p604800/cC0#14.52/30.28196/-97.73198), it should navigate you to a map of the surrounding University of Texas area. If you click on the sensor currently located in the DKR Texas Memorial Stadium, you'll find that the name of that particular sensor is "PA_II_D8B6". 

Let's take a closer look at the UT Stadium Sensor! In the dataframe below we filter the dataframe by the sensor name ("PA_II_D8B6") to pick out the row that corresponds to the specific sensor we are looking for. 


In [7]:
hide_me
df[df['name'] == "PA_II_D8B6"]

,parent,lat,lon,name,location_type,pm_2.5,temp_f,temp_c,humidity,pressure,...,last_update_check,created,uptime,is_owner,10min_avg,30min_avg,1hour_avg,6hour_avg,1day_avg,1week_avg
id,,,,,,,,,,,,,,,,,,,,,
27395,None,30.285069,-97.732877,PA_II_D8B6,outside,11.2,63.0,17.222222,50.0,995.28,...,None,None,None,False,10.08,11.01,11.11,7.41,8.99,10.01


<br>

The row above gives us loads of information on the state of the AQI in the UT Stadium at the present moment, but it would be nice to see the AQI information over time. Below is a dataframe that contains information about the UT Stadium sensor roughly over the last 7 days. We can do this by filtering the times each entry was created at.

In [8]:
hide_me

## data from PA_II_D8B6 (UT Stadium Sensor) sensor from the past week
from purpleair.sensor import Sensor
se = Sensor(27395)
UT_stadium = se.parent.get_historical(weeks_to_get=1,thingspeak_field='secondary')
UT_stadium['Date'] = [i.date().strftime("%d-%b-%Y") for i in UT_stadium['created_at']]
UT_stadium

,created_at,0.3um/dl,0.5um/dl,1.0um/dl,2.5um/dl,5.0um/dl,10.0um/dl,PM1.0 (CF=ATM) ug/m3,PM10 (CF=ATM) ug/m3,Date
entry_id,,,,,,,,,,
558924,2021-11-30 00:03:06+00:00,501.63,145.04,18.42,1.68,0.00,0.00,1.89,2.88,30-Nov-2021
558925,2021-11-30 00:05:08+00:00,596.52,158.91,17.93,2.68,0.71,0.46,2.18,3.95,30-Nov-2021
558926,2021-11-30 00:07:14+00:00,689.04,190.02,24.50,1.46,0.75,0.00,2.80,4.79,30-Nov-2021
558927,2021-11-30 00:09:09+00:00,577.45,160.88,19.90,0.97,0.00,0.00,2.09,3.14,30-Nov-2021
558928,2021-11-30 00:11:09+00:00,527.00,147.93,18.98,1.72,0.95,0.00,2.04,3.47,30-Nov-2021
...,...,...,...,...,...,...,...,...,...,...
562417,2021-12-06 23:48:26+00:00,240.21,71.17,12.10,1.52,0.00,0.00,0.59,1.38,06-Dec-2021
562418,2021-12-06 23:50:23+00:00,246.98,66.59,11.55,0.93,0.93,0.48,0.55,1.79,06-Dec-2021
562419,2021-12-06 23:52:21+00:00,298.55,86.48,10.75,1.39,0.21,0.21,0.70,1.73,06-Dec-2021


As you can see from the "created_at" column, the AQI was taken every two minutes over the past 7 days. The data frame also contains information on PM paticules of different diameters such as 0.3, 0.5, 1.0, 2.5, 5.0 and 10.0.

<br>

While this dataframe is useful, there are too many rows of data (~4000) to look at! Below is a widget that plots a line graph of the PM 2.5 measure over a specific day. 

**The drop down bar allows you to pick which day you would like graphed, so go ahead and pick a day!**

In [9]:
hide_me

def f(date):
    fig = plt.figure(figsize=(20,3))
    plt.plot(UT_stadium['created_at'].loc[UT_stadium['Date'] == date], UT_stadium["2.5um/dl"].loc[UT_stadium['Date'] == date])
    plt.xlabel('Time')
    plt.ylabel('PM 2.5 Particle Count')
    plt.title('UT Stadium Sensor PM 2.5')
    plt.rcParams["figure.figsize"] = (20,3)
    
interact(f, date = list(UT_stadium['Date'].unique()));

interactive(children=(Dropdown(description='date', options=('30-Nov-2021', '01-Dec-2021', '02-Dec-2021', '03-D…

The line plots above displays the date and hour along the x-axis and the PM 2.5 Particle count along the y-axis.

<br>

**QUESTION: What trends do you notice about the line plot?**



*Your answer here*

<br>

While the line plots do show us a trend in the PM2.5 count over time, we still have not clue how that translates to the API Index. The next section will discuss what AQI is and how it is calculated.

### AQI Index
The AQI Index contains 6 categories that air quality can fall into. Each category contains a range of index values from 0 - 500 that is calculated from the regions PM 2.5 measure. The chart below is provided by the US Environmental Protection Agency (EPA) and shows the official AQI Index (these breakpoints were revised in 2012). 

For more information on how AQI Index is calculated, take a look at the AQI Index Factsheet provided by the EPA [here](https://www.epa.gov/sites/default/files/2016-04/documents/2012_aqi_factsheet.pdf)!

<p align="center">
  <img src="images/AQI-category.png" width="" height="" align="center">
</p>

Now that we know how sesors work, what they measure and how AQI Indexes are calculated, let's see if we can create a visualization of AQI Indexes that are a little closer to home!

First, let's find a group of sensors that are in Texas. We use a range of longitude and latitude coordinates to decide whether to include or exclude a sensor. Below is a dataframe of sensors in Texas.


In [10]:
hide_me

TX_data = df.loc[(df["lat"] >= 25.9) & (df["lat"] <= 34.1) & (df["lon"] >= -104.9) & (df["lon"] <= -93.1)]
TX_data = TX_data[["lat", "lon", "name", "location_type", "pm_2.5", "temp_f", "humidity", "pressure"]]
TX_data

,lat,lon,name,location_type,pm_2.5,temp_f,humidity,pressure
id,,,,,,,,
25999,30.053808,-95.494643,Villages of Bridgestone AQI,outside,14.50,69.0,61.0,1011.57
74265,29.939444,-95.671354,16815 Flower Mist Lane,inside,3.89,78.0,33.0,1012.22
30327,33.346098,-96.355190,"236 County Road 5020, Blue Ridge, Texas 75424-...",outside,6.43,NaN,NaN,NaN
83065,30.246557,-98.065587,301 Barton Creek Dr,outside,9.54,58.0,48.0,976.82
98139,29.818880,-95.695667,"3623 Shadow Trail, Houston, Texas 77084",outside,15.68,71.0,57.0,1012.12
...,...,...,...,...,...,...,...,...
34431,29.762515,-95.465982,WPI-2,inside,0.49,80.0,40.0,1011.13
27629,33.538329,-101.781656,Yellow House Canyon,outside,4.89,45.0,33.0,905.53
65197,30.261005,-97.770814,Zilker #1,outside,13.25,58.0,54.0,997.93


<br>

Now that we have a smaller subset of data to work with, the next step is to use the PM 2.5 measures to assign each sensor to an AQI Index Category and corresponding color. We can do this by making a list of color codes and assigning a column in our data frame to that list.

In [11]:
hide_me

#creating a column that indicates the AQI code name
color_code = []
for i in TX_data["pm_2.5"].to_list():
    if i <= 12.0:
        color_code.append('green')
    elif (i < 12) & (i <=35.4):
        color_code.append('yellow')
    elif (i < 35.5) & (i <=55.4):
        color_code.append('orange')   
    elif (i < 55.5) & (i <=150.4):
        color_code.append('red')
    elif (i < 150.5) & (i <=250.4):
        color_code.append('purple')
    else:
        color_code.append('darkpurple')

TX_data['code'] = color_code

<br>

Our last step is to use the longitude and latitude coordinates to map the relative location of the sensor with is corresponding AQI Index color! The widget below contains two sliders. One represents the Latitude value and the other is the Longitude value. 

**Slide the sliders left and right to display a mapping of the sensors in that latitude and longitude region, or use your cursor to drag the mapping area.**

**Hint: Houston, TX - Lat: 29.7604 / Lon: -95.3698** 

In [12]:
hide_me

def map(Latitude ,Longitude):
    m = folium.Map(width=600, height=500, location=[Latitude, Longitude])
    
    for i in np.arange(len(TX_data) - 1):
        folium.Marker(
            location=[TX_data.iloc[i]['lat'], TX_data.iloc[i]['lon']],
            popup=TX_data.iloc[i]['name'],
            icon=folium.Icon(color=TX_data.iloc[i]['code']),
        ).add_to(m)
    display(m)
    
interact(map, Latitude = (26, 34, 0.001) , Longitude = (-103, -93, 0.001));
## Houston, TX - Lat: 29.7604 / Lon: -95.3698

interactive(children=(FloatSlider(value=30.0, description='Latitude', max=34.0, min=26.0, step=0.001), FloatSl…

Now that we have created a map we can easily see what the AQI index is across the city! 

<br>

**QUESTION: What do you notice about the map?**

*Your answer here*

<br>

Developed By: Melisa Esqueda, Maham Bawaney & Karalyn Chong